In [3]:
# 📌 STEP 1: Import Required Libraries

import os
import torch
import pandas as pd
from transformers import AutoTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# ✅ Ensure GPU is Used if Available
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Set Random Seed for Reproducibility
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Using device: cuda


In [4]:
# 📌 STEP 2: load the Dataset
train_path = "/home/jupyter-st124872/NLP/A7/train.csv"
test_path = "/home/jupyter-st124872/NLP/A7/test.csv"

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

# ✅ Check the Dataset Structure
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
# 📌 STEP 3: Preprocessing the Jigsaw Toxic Comment Dataset

# ✅ Load DistilBERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# ✅ Define Text Preprocessing Function
def preprocess_text(text):
    """
    Cleans and tokenizes text for model training.
    - Converts to lowercase
    - Removes special characters
    - Strips extra spaces
    """
    text = str(text).lower()
    text = text.replace("\n", " ").replace("\r", " ")  # Remove newlines and carriage returns
    text = text.strip()  # Remove leading/trailing spaces
    return text

# ✅ Apply Preprocessing to Dataset
df_train["cleaned_text"] = df_train["comment_text"].astype(str).apply(preprocess_text)

# ✅ Convert Multi-Label Classification to Binary Classification (Toxic vs. Non-Toxic)
label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
df_train["toxic_binary"] = (df_train[label_columns].sum(axis=1) > 0).astype(int)

# ✅ Define Tokenization Function
def tokenize_function(examples):
    """
    Tokenizes text using DistilBERT tokenizer.
    - Applies padding, truncation
    - Ensures max length of 128 tokens
    """
    return tokenizer(examples["cleaned_text"].tolist(), padding="max_length", truncation=True, max_length=128)

# ✅ Apply Tokenization to Train Data
train_encodings = tokenize_function(df_train)

# ✅ Confirm Preprocessing Done
df_train.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned_text,toxic_binary
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation why the edits made under my userna...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww! he matches this background colour i'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"hey man, i'm really not trying to edit war. it...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,""" more i can't make any real suggestions on im...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"you, sir, are my hero. any chance you remember...",0


In [6]:
# 📌 TASK 4: Split Data for Training & Validation

# ✅ Step 1: Split df_train into Train (80%) and Validation (20%)
train_df, valid_df = train_test_split(df_train, test_size=0.2, random_state=SEED, stratify=df_train["toxic_binary"])

# ✅ Step 2: Tokenize Validation Data
valid_encodings = tokenize_function(valid_df)

# ✅ Define Custom PyTorch Dataset Class
class ToxicCommentsDataset(Dataset):
    """
    Custom PyTorch Dataset for the Jigsaw Toxic Comment Classification Task.
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# ✅ Step 3: Convert Encoded Data into PyTorch Dataset
train_dataset = ToxicCommentsDataset(train_encodings, train_df["toxic_binary"].tolist())
valid_dataset = ToxicCommentsDataset(valid_encodings, valid_df["toxic_binary"].tolist())

# ✅ Confirm Dataset Sizes
len(train_dataset), len(valid_dataset)


(127656, 31915)

In [7]:
# ✅ Debug Step 1: Check DataFrame Columns
print("Columns in train_df:", train_df.columns.tolist())
print("Columns in valid_df:", valid_df.columns.tolist())

# ✅ Debug Step 2: Check the Structure of Encodings
print("Train Encoding Keys:", train_encodings.keys())
print("Valid Encoding Keys:", valid_encodings.keys())

# ✅ Debug Step 3: Ensure 'toxic_binary' Exists in Both Datasets
print("Toxic Binary Column in Train:", "toxic_binary" in train_df.columns)
print("Toxic Binary Column in Valid:", "toxic_binary" in valid_df.columns)


Columns in train_df: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'cleaned_text', 'toxic_binary']
Columns in valid_df: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'cleaned_text', 'toxic_binary']
Train Encoding Keys: dict_keys(['input_ids', 'attention_mask'])
Valid Encoding Keys: dict_keys(['input_ids', 'attention_mask'])
Toxic Binary Column in Train: True
Toxic Binary Column in Valid: True


In [6]:
# 📌 TASK 5: Train DistilBERT Student Models (Odd & Even Layers)

# ✅ Reload Pretrained DistilBERT Model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# ✅ Define Function to Create Student Model with Selected Layers
def create_student_model(selected_layers):
    """
    Creates a student DistilBERT model with a subset of layers from the original teacher model.
    """
    student_config = model.config
    student_config.num_hidden_layers = len(selected_layers)
    student_model = DistilBertForSequenceClassification(student_config)

    # Copy selected layers from the teacher model
    student_model.distilbert.transformer.layer = torch.nn.ModuleList(
        [model.distilbert.transformer.layer[i] for i in selected_layers]
    )

    return student_model

# ✅ Create Student Models (Using Odd and Even Layers)
odd_layers = [1, 3, 5]  # ✅ Corrected for 6-layer DistilBERT
even_layers = [0, 2, 4]  # ✅ Corrected for 6-layer DistilBERT

odd_model = create_student_model(odd_layers)
even_model = create_student_model(even_layers)

# ✅ Define Training Arguments (Ensuring `eval_dataset` is Included)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # ✅ Ensures evaluation every epoch
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
)

# ✅ Trainer Initialization for Odd Model
trainer_odd = Trainer(
    model=odd_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,  # ✅ Added validation dataset
)

# ✅ Trainer Initialization for Even Model
trainer_even = Trainer(
    model=even_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,  # ✅ Added validation dataset
)

# ✅ Train Odd-Layer Student Model
print("Training Odd-Layer Student Model...")
trainer_odd.train()

# ✅ Train Even-Layer Student Model
print("Training Even-Layer Student Model...")
trainer_even.train()

# ✅ Save Models for Evaluation
odd_model.save_pretrained("./odd_layer_student_model")
even_model.save_pretrained("./even_layer_student_model")

# ✅ Training Complete
print("✅ Training for Odd and Even Layer Models is complete!")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training Odd-Layer Student Model...


Epoch,Training Loss,Validation Loss
1,0.339400,0.338872
2,0.321700,0.330867
3,0.320500,0.359328


Training Even-Layer Student Model...


Epoch,Training Loss,Validation Loss
1,0.340300,0.340194
2,0.321800,0.344678
3,0.306100,0.377218


✅ Training for Odd and Even Layer Models is complete!


In [1]:
import loralib as lora
import torch
from transformers import DistilBertForSequenceClassification

# ✅ Load DistilBERT
base_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# ✅ Replace q_lin and v_lin with LoRA-compatible layers
for layer in base_model.distilbert.transformer.layer:
    d_in = layer.attention.q_lin.in_features
    d_out = layer.attention.q_lin.out_features
    layer.attention.q_lin = lora.Linear(d_in, d_out, r=8)
    layer.attention.v_lin = lora.Linear(d_in, d_out, r=8)

# ✅ Freeze all other weights except LoRA
lora.mark_only_lora_as_trainable(base_model)

# ✅ Show which params are trainable
for name, param in base_model.named_parameters():
    if param.requires_grad:
        print(name)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


distilbert.transformer.layer.0.attention.q_lin.lora_A
distilbert.transformer.layer.0.attention.q_lin.lora_B
distilbert.transformer.layer.0.attention.v_lin.lora_A
distilbert.transformer.layer.0.attention.v_lin.lora_B
distilbert.transformer.layer.1.attention.q_lin.lora_A
distilbert.transformer.layer.1.attention.q_lin.lora_B
distilbert.transformer.layer.1.attention.v_lin.lora_A
distilbert.transformer.layer.1.attention.v_lin.lora_B
distilbert.transformer.layer.2.attention.q_lin.lora_A
distilbert.transformer.layer.2.attention.q_lin.lora_B
distilbert.transformer.layer.2.attention.v_lin.lora_A
distilbert.transformer.layer.2.attention.v_lin.lora_B
distilbert.transformer.layer.3.attention.q_lin.lora_A
distilbert.transformer.layer.3.attention.q_lin.lora_B
distilbert.transformer.layer.3.attention.v_lin.lora_A
distilbert.transformer.layer.3.attention.v_lin.lora_B
distilbert.transformer.layer.4.attention.q_lin.lora_A
distilbert.transformer.layer.4.attention.q_lin.lora_B
distilbert.transformer.layer

In [3]:
# ✅ Install LoRA library 
# !pip install loralib

# ✅ Imports
import torch
import loralib as lora
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# ✅ Set CPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("🚀 Using device:", device)

# ✅ STEP 1: Load Dataset
df = pd.read_csv("/home/jupyter-st124872/NLP/A7/train.csv")
df["cleaned_text"] = df["comment_text"].astype(str).str.lower().str.replace(r"[^a-zA-Z0-9\s]", " ", regex=True)
label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
df["label"] = (df[label_cols].sum(axis=1) > 0).astype(int)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["cleaned_text"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)

# ✅ STEP 2: Tokenize
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)

# ✅ STEP 3: Dataset Class
class ToxicDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = ToxicDataset(train_encodings, train_labels)
val_dataset = ToxicDataset(val_encodings, val_labels)

# ✅ STEP 4: Load DistilBERT and Inject LoRA
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

for layer in model.distilbert.transformer.layer:
    d_in = layer.attention.q_lin.in_features
    d_out = layer.attention.q_lin.out_features
    layer.attention.q_lin = lora.Linear(d_in, d_out, r=8)
    layer.attention.v_lin = lora.Linear(d_in, d_out, r=8)

# ✅ Make only LoRA layers trainable
lora.mark_only_lora_as_trainable(model)
model.to(device)

# ✅ STEP 5: Optimizer & Dataloader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

optimizer = AdamW(model.parameters(), lr=5e-5)

# ✅ STEP 6: Training Loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} - Avg Train Loss: {avg_loss:.4f}")

# ✅ STEP 7: Save LoRA-only state_dict
torch.save(lora.lora_state_dict(model), "lora_distilbert_cpu.pt")
print("✅ LoRA training complete and checkpoint saved.")


🚀 Using device: cuda


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/jupyter-st124872/.local/lib/python3.12/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|██████████| 7979/7979 [06:07<00:00, 21.72it/s]


✅ Epoch 1 - Avg Train Loss: 0.2348


Epoch 2: 100%|██████████| 7979/7979 [06:30<00:00, 20.42it/s]


✅ Epoch 2 - Avg Train Loss: 0.1462


Epoch 3: 100%|██████████| 7979/7979 [06:35<00:00, 20.20it/s]

✅ Epoch 3 - Avg Train Loss: 0.1227
✅ LoRA training complete and checkpoint saved.


In [8]:
import torch
import loralib as lora
from transformers import DistilBertForSequenceClassification
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

# ✅ Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"🚀 Evaluating on device: {device}")

# ✅ Load Odd/Even Models (standard loading)
odd_model = DistilBertForSequenceClassification.from_pretrained("./odd_layer_student_model").to(device)
even_model = DistilBertForSequenceClassification.from_pretrained("./even_layer_student_model").to(device)

# ✅ Load LoRA Model (inject LoRA layers + load LoRA weights)
lora_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
for layer in lora_model.distilbert.transformer.layer:
    d_in = layer.attention.q_lin.in_features
    d_out = layer.attention.q_lin.out_features
    layer.attention.q_lin = lora.Linear(d_in, d_out, r=8)
    layer.attention.v_lin = lora.Linear(d_in, d_out, r=8)

# Load LoRA adapter-only weights (strict=False is essential!)
lora_model.load_state_dict(torch.load("lora_distilbert_cpu.pt"), strict=False)
lora_model.to(device)

# ✅ Define Evaluation Function
def evaluate_model(model, dataset):
    model.eval()
    predictions, true_labels = [], []

    for batch in DataLoader(dataset, batch_size=16):
        inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=-1).cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(batch["labels"].cpu().numpy())

    acc = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions, target_names=["Non-Toxic", "Toxic"], digits=4)
    conf_matrix = confusion_matrix(true_labels, predictions)
    return acc, report, conf_matrix

# ✅ Evaluate All Three Models
print("\n🔍 Evaluating Odd-Layer Student Model...")
odd_acc, odd_report, odd_conf_matrix = evaluate_model(odd_model, valid_dataset)

print("\n🔍 Evaluating Even-Layer Student Model...")
even_acc, even_report, even_conf_matrix = evaluate_model(even_model, valid_dataset)

print("\n🔍 Evaluating LoRA Model...")
lora_acc, lora_report, lora_conf_matrix = evaluate_model(lora_model, valid_dataset)

# ✅ Display Accuracy Scores
results_df = pd.DataFrame({
    "Model": ["Odd-Layer", "Even-Layer", "LoRA"],
    "Accuracy": [odd_acc, even_acc, lora_acc]
})
print("\n📊 Model Accuracy Scores:")
print(results_df)

# ✅ Show Classification Reports
print("\n📄 Odd-Layer Model Report:\n", odd_report)
print("\n📄 Even-Layer Model Report:\n", even_report)
print("\n📄 LoRA Model Report:\n", lora_report)

# ✅ Optional: View Confusion Matrices
print("\n🧩 Odd Confusion Matrix:\n", odd_conf_matrix)
print("\n🧩 Even Confusion Matrix:\n", even_conf_matrix)
print("\n🧩 LoRA Confusion Matrix:\n", lora_conf_matrix)


🚀 Evaluating on device: cuda


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔍 Evaluating Odd-Layer Student Model...


/home/jupyter-st124872/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-st124872/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-st124872/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r


🔍 Evaluating Even-Layer Student Model...

🔍 Evaluating LoRA Model...

📊 Model Accuracy Scores:
        Model  Accuracy
0   Odd-Layer  0.898324
1  Even-Layer  0.896130
2        LoRA  0.361491

📄 Odd-Layer Model Report:
               precision    recall  f1-score   support

   Non-Toxic     0.8983    1.0000    0.9464     28670
       Toxic     0.0000    0.0000    0.0000      3245

    accuracy                         0.8983     31915
   macro avg     0.4492    0.5000    0.4732     31915
weighted avg     0.8070    0.8983    0.8502     31915


📄 Even-Layer Model Report:
               precision    recall  f1-score   support

   Non-Toxic     0.8985    0.9970    0.9452     28670
       Toxic     0.1635    0.0052    0.0102      3245

    accuracy                         0.8961     31915
   macro avg     0.5310    0.5011    0.4777     31915
weighted avg     0.8238    0.8961    0.8501     31915


📄 LoRA Model Report:
               precision    recall  f1-score   support

   Non-Toxic     0.